# LHS is not available on TS machines and will be performed elsewhere

# calculate the stress state of the reference block

### For the JD_Sula_2005_gmc grid, there are 10 k layers. Reservoir starts at k=6. Use block (50, 1, 6) as reference block for *STRESSGRAD calculation. Its grid top = 670.7188 m and bottom = 671.9521 m.

In [1]:
grid_top = 670.7188
grid_bottom = 671.9521
grid_ave = (grid_top + grid_bottom)/2
print(grid_ave)

671.33545


In [3]:
# This code has been verified using the StressTransform3D function
import numpy as np
SH=25
Sh=14.6
beta = 300-90 #rotate from SH to x axis
cos_2beta = np.cos(np.radians(2*beta))
sin_2beta = np.sin(np.radians(2*beta))
sigma_x_grad=(SH+Sh)/2 + (SH-Sh)/2*cos_2beta
sigma_y_grad=(SH+Sh)/2 - (SH-Sh)/2*cos_2beta
tau_xy_grad = (SH-Sh)/2*sin_2beta
print('sigma_x gradient =',sigma_x_grad)
print('sigma_y gradient =',sigma_y_grad)
print('tau_xy gradient =',tau_xy_grad)

sigma_x gradient = 22.4
sigma_y gradient = 17.200000000000003
tau_xy gradient = 4.503332099679082


In [7]:
print('For the reference block, the effective stresses are:')
print('sigma_x = ', (sigma_x_grad - 10) * grid_ave, ' kPa')
print('sigma_y = ', (sigma_y_grad - 10) * grid_ave, ' kPa')
print('sigma_z = ', (22.7 - 10) * grid_ave, ' kPa')
print('tau_xy = ', tau_xy_grad * grid_ave, ' kPa')

For the reference block, the effective stresses are:
sigma_x =  8324.55958  kPa
sigma_y =  4833.615240000002  kPa
sigma_z =  8525.960215  kPa
tau_xy =  3023.246481637502  kPa


### check one point: pretty close

In [20]:
grid_top2 = 685.8252
grid_bottom2 = 686.126
grid_ave2 = (grid_top2 + grid_bottom2)/2

print('For the checking block, the effective stresses are:')
print('sigma_x = ', (sigma_x_grad - 10) * grid_ave2, ' kPa')
print('sigma_y = ', (sigma_y_grad - 10) * grid_ave2, ' kPa')
print('sigma_z = ', (22.7 - 10) * grid_ave2, ' kPa')
print('tau_xy = ', tau_xy_grad * grid_ave2, ' kPa')

For the checking block, the effective stresses are:
sigma_x =  8506.09744  kPa
sigma_y =  4939.024320000002  kPa
sigma_z =  8711.89012  kPa
tau_xy =  3089.1759390766183  kPa


In [21]:
grid_top2 = 2503.424
grid_bottom2 = 2596.873
grid_ave2 = (grid_top2 + grid_bottom2)/2

print('For the checking block, the effective stresses are:')
print('sigma_x = ', (sigma_x_grad - 10) * grid_ave2, ' kPa')
print('sigma_y = ', (sigma_y_grad - 10) * grid_ave2, ' kPa')
print('sigma_z = ', (22.7 - 10) * grid_ave2, ' kPa')
print('tau_xy = ', tau_xy_grad * grid_ave2, ' kPa')

For the checking block, the effective stresses are:
sigma_x =  31621.8414  kPa
sigma_y =  18361.06920000001  kPa
sigma_z =  32386.88595  kPa
tau_xy =  11484.165598998463  kPa


## PORO/PERMX pairs are NOT sampled more than once (from file names instead of files)

In [16]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from pathlib import Path
import sys

# --------------------------
# Setup for sampling
# --------------------------
params = ['E_GPa', 'PR', 'SH_MPa/km', 'Sh_MPa/km', 'Sv_MPa/km', 'SH_azi_deg']
l_bounds = [15e6, 0.2, 25 * 0.9, 14.6 * 0.9, 22.7 * 0.9, 290]
u_bounds = [25e6, 0.4, 25 * 1.1, 14.6 * 1.1, 22.7 * 1.1, 310]
num_samples = 6  # Change as needed

# --------------------------
# Load PORO and PERMX file names
# --------------------------
property_file_names = np.load('property_file_names.npy')

# poro_files = sorted([f for f in data_dir.iterdir() if "PORO" in f.name])
# permx_files = sorted([f for f in data_dir.iterdir() if "PERMX" in f.name])
poro_file_names = sorted([name for name in property_file_names if "PORO" in name.upper()])
permx_file_names = sorted([name for name in property_file_names if "PERMX" in name.upper()])

if not poro_file_names or not permx_file_names:
    print("Error: PORO or PERMX file names not found.")
    sys.exit(1)

if len(poro_file_names) != len(permx_file_names):
    raise ValueError(f"Number of PORO file names ({len(poro_file_names)}) does not match number of PERMX file names ({len(permx_file_names)})")

num_pairs = len(poro_file_names)

if num_samples > num_pairs:
    raise ValueError(f"Cannot sample {num_samples} unique poro/permx pairs: only {num_pairs} available.")

# --------------------------
# Latin Hypercube Sampling for parameters
# --------------------------
sampler = qmc.LatinHypercube(d=len(params))
sample = sampler.random(n=num_samples)
sample_params = qmc.scale(sample, l_bounds, u_bounds)
df_params = pd.DataFrame(np.round(sample_params, 2), columns=params)

# --------------------------
# Store poro/permx pairs
# --------------------------
df_params["PORO_file"] = [str(poro_file_names[i]) for i in range(num_samples)]
df_params["PERMX_file"] = [str(permx_file_names[i]) for i in range(num_samples)]

# --------------------------
# add prefix to file names
# --------------------------
prefix = "data_properties/"
df_params["PORO_file"] = df_params["PORO_file"].apply(lambda x: f"{prefix}{x}")
df_params["PERMX_file"] = df_params["PERMX_file"].apply(lambda x: f"{prefix}{x}")

# --------------------------
# Calculate stress state parameters
# --------------------------
df_params['beta'] = df_params['SH_azi_deg'] - 90  # Rotate from SH to x-axis
df_params['cos_2beta'] = np.cos(np.radians(2 * df_params['beta']))
df_params['sin_2beta'] = np.sin(np.radians(2 * df_params['beta']))

df_params['sigma_x'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 + \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
df_params['sigma_y'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 - \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
df_params['tau_xy'] = (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['sin_2beta']

# --------------------------
# Output
# --------------------------
df_params.to_csv("sampled_parameters_and_files.csv", index=False)
df_params



,E_GPa,PR,SH_MPa/km,Sh_MPa/km,Sv_MPa/km,SH_azi_deg,PORO_file,PERMX_file,beta,cos_2beta,sin_2beta,sigma_x,sigma_y,tau_xy
0,18885211.12,0.30,23.13,13.53,21.49,290.62,data_properties/JD_BASECASE_21_PORO.dat,data_properties/JD_BASECASE_21_PERMX.dat,200.62,0.751955,0.659215,21.939383,14.720617,3.164230
1,15265605.61,0.27,26.72,14.51,23.95,299.76,data_properties/JD_BASECASE_22_PORO.dat,data_properties/JD_BASECASE_22_PERMX.dat,209.76,0.507238,0.861806,23.711685,17.518315,5.261327
2,20341604.68,0.36,25.21,15.25,23.20,304.32,data_properties/JD_BASECASE_26_PORO.dat,data_properties/JD_BASECASE_26_PERMX.dat,214.32,0.364227,0.931310,22.043849,18.416151,4.637925
3,22038429.14,0.37,24.22,14.76,22.49,296.51,data_properties/JD_BASECASE_27_PORO.dat,data_properties/JD_BASECASE_27_PERMX.dat,206.51,0.601536,0.798846,22.335266,16.644734,3.778539
4,23601835.10,0.22,23.59,15.65,24.67,307.34,data_properties/JD_BASECASE_28_PORO.dat,data_properties/JD_BASECASE_28_PERMX.dat,217.34,0.264210,0.964465,20.668913,18.571087,3.828927
5,16882215.04,0.24,25.91,13.68,20.91,302.72,data_properties/JD_BASECASE_38_PORO.dat,data_properties/JD_BASECASE_38_PERMX.dat,212.72,0.415646,0.909527,22.336675,17.253325,5.561755


In [11]:
property_file_names = np.load('property_file_names.npy')
print(property_file_names)

['JD_BASECASE_21_PERMX.dat' 'JD_BASECASE_21_PORO.dat'
 'JD_BASECASE_22_PERMX.dat' 'JD_BASECASE_22_PORO.dat'
 'JD_BASECASE_26_PERMX.dat' 'JD_BASECASE_26_PORO.dat'
 'JD_BASECASE_27_PERMX.dat' 'JD_BASECASE_27_PORO.dat'
 'JD_BASECASE_28_PERMX.dat' 'JD_BASECASE_28_PORO.dat'
 'JD_BASECASE_38_PERMX.dat' 'JD_BASECASE_38_PORO.dat']


In [14]:
property_file_names[11]

'JD_BASECASE_38_PORO.dat'

In [4]:
poro_file_names = sorted([name for name in property_file_names if "PORO" in name.upper()])
permx_file_names = sorted([name for name in property_file_names if "PERMX" in name.upper()])

print(poro_file_names)
print(permx_file_names)

['JD_BASECASE_21_PORO.dat', 'JD_BASECASE_22_PORO.dat', 'JD_BASECASE_26_PORO.dat', 'JD_BASECASE_27_PORO.dat', 'JD_BASECASE_28_PORO.dat', 'JD_BASECASE_38_PORO.dat']
['JD_BASECASE_21_PERMX.dat', 'JD_BASECASE_22_PERMX.dat', 'JD_BASECASE_26_PERMX.dat', 'JD_BASECASE_27_PERMX.dat', 'JD_BASECASE_28_PERMX.dat', 'JD_BASECASE_38_PERMX.dat']
